In [1]:
import plotly.express as px
import pandas as pd

In [2]:
df = pd.read_csv("data-cleaning\output\google_app_scrap_cleaned.csv")
df.head()

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,...,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,"""Ghost Voice Catcher"" AUTO EVP",3.933333,['Lifestyle'],topselling_paid,68,12,1,6,7,42,...,1100000.0,BIG BEARD Studios,114 1/2 Ash st. Nashua NH 03060,Usa,Everyone,False,False,False,1820,3482
1,"""OXXO""",4.740000,['Puzzle'],topselling_paid,975,23,0,15,94,843,...,81000000.0,Hamster On Coke Games,ul Jugosłowiańska 37F/10\r\n73-110 Stargard Sz...,Poland,Everyone,False,False,False,0,945
2,#DRIVE,4.434152,['Racing'],topgrossing,229679,9352,4740,15249,47804,152534,...,132000000.0,Pixel Perfect Dude S.A.,ul. Polna 28/6 lok. 4\r\n62-800 Kalisz\r\nPolska,NaN,Everyone,True,True,False,4,1084
3,#SelfCare,4.463476,['Role Playing'],topselling_free,14932,865,336,1088,1353,11290,...,83000000.0,TRU LUV Inc.,"186-720 King Street West\r\nToronto, Ontario, ...",Canada,Teen,False,True,False,952,1275
4,#open Polyamorous + ENM Dating,2.550000,['Dating'],topselling_free,708,263,163,63,70,149,...,19000000.0,#open,NaN,NaN,Mature 17+,False,True,False,17,1026


Factors determining if an app is good:
   1. Max installs
   2. Rating
   
    
Approach 1:
Only comparing the average (mean & median) of max installs of the apps developed by each country.

Approach 2:
Only comparing the average mean of rating of the apps developed by each country.

Approach 3:
Take the top 10 apps of each country in max installs.
Find the mean of the rating of this 10 apps.
Compare the mean ratings.

Approach 4:
Take the top 10 apps of each country in max installs.
Find the mean median of the max installs of this 10 apps.
Compare the max installs.

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17470 entries, 0 to 17469
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             17470 non-null  object 
 1   RATING               17470 non-null  float64
 2   CATEGORY             17470 non-null  object 
 3   COLLECTIONS          17470 non-null  object 
 4   RATING_COUNT         17470 non-null  int64  
 5   1_STAR_RATINGS       17470 non-null  int64  
 6   2_STAR_RATINGS       17470 non-null  int64  
 7   3_STAR_RATINGS       17470 non-null  int64  
 8   4_STAR_RATINGS       17470 non-null  int64  
 9   5_STAR_RATINGS       17470 non-null  int64  
 10  REVIEW_COUNT         17470 non-null  float64
 11  INSTALLS             17470 non-null  object 
 12  MIN_INSTALLS         17470 non-null  int64  
 13  MAX_INSTALLS         17470 non-null  int64  
 14  FREE                 17470 non-null  bool   
 15  PRICE                17470 non-null 

Dropping apps that has NULL values for country

In [3]:
edited_df = df.dropna(subset=['COUNTRY'])
edited_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11582 entries, 0 to 17468
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   APP_NAME             11582 non-null  object 
 1   RATING               11582 non-null  float64
 2   CATEGORY             11582 non-null  object 
 3   COLLECTIONS          11582 non-null  object 
 4   RATING_COUNT         11582 non-null  int64  
 5   1_STAR_RATINGS       11582 non-null  int64  
 6   2_STAR_RATINGS       11582 non-null  int64  
 7   3_STAR_RATINGS       11582 non-null  int64  
 8   4_STAR_RATINGS       11582 non-null  int64  
 9   5_STAR_RATINGS       11582 non-null  int64  
 10  REVIEW_COUNT         11582 non-null  float64
 11  INSTALLS             11582 non-null  object 
 12  MIN_INSTALLS         11582 non-null  int64  
 13  MAX_INSTALLS         11582 non-null  int64  
 14  FREE                 11582 non-null  bool   
 15  PRICE                11582 non-null 

### Approach 1

Taking the mean of MAX_INSTALLS for each country

In [5]:
max_installs_by_mean = pd.DataFrame(edited_df.groupby(["COUNTRY"])["MAX_INSTALLS"].agg("mean"))
max_installs_by_mean.sort_values("MAX_INSTALLS", ascending = False)
max_installs_by_mean = max_installs_by_mean.reset_index()
max_installs_by_mean.rename(columns={'MAX_INSTALLS': 'MEAN_MAX_INSTALLS'}, inplace = True)
max_installs_by_mean

,COUNTRY,MEAN_MAX_INSTALLS
0,Albania,2.054479e+07
1,Algeria,1.629410e+05
2,Andorra,1.050528e+07
3,Argentina,1.702661e+07
4,Armenia,7.473048e+06
...,...,...
98,Uzbekistan,2.642840e+06
99,Venezuela,8.762000e+03
100,Vietnam,8.806995e+06
101,Yemen,1.031400e+04


Taking only the top 10 countries in MEAN_MAX_INSTALLS

In [6]:
max_installs_by_mean = max_installs_by_mean.nlargest(10,['MEAN_MAX_INSTALLS'])
max_installs_by_mean

,COUNTRY,MEAN_MAX_INSTALLS
47,Luxembourg,4.879105e+08
62,Oman,6.050167e+07
26,Finland,6.021174e+07
81,South Korea,5.321177e+07
77,Singapore,5.043485e+07
97,Usa,4.871036e+07
21,Denmark,4.643685e+07
61,Norway,3.377678e+07
37,Ireland,2.922904e+07
19,Cyprus,2.620449e+07


Plotting a bar plot for MEAN_MAX_INSTALLS against COUNTRY

In [9]:
fig = px.bar(max_installs_by_mean, x='COUNTRY', y='MEAN_MAX_INSTALLS')
fig.show()

It seems that the MEAN_MAX_INSTALLS of Luxembourg has a very large discrepancy compared to all the other countries.

Plotting a the MEAN_MAX_INSTALLS of the 10 countries in a map

In [8]:
fig = px.scatter_geo(max_installs_by_mean, locations="COUNTRY", 
                     hover_name="COUNTRY", size = 'MEAN_MAX_INSTALLS',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()

Exploring Luxembourg

In [11]:
luxembourg = pd.DataFrame(df[df['COUNTRY'] == 'Luxembourg'])
luxembourg = luxembourg[['APP_NAME','MAX_INSTALLS']]
fig = px.box(luxembourg, x="MAX_INSTALLS")
fig.show()

In [12]:
fig = px.bar(luxembourg, x="APP_NAME", y="MAX_INSTALLS")
fig.show()

The two plots above show that Luxembourg has apps that have very large discrepancy. The MAX_INSTALLS of Skype and Viber are the two extreme values in this case.

In [15]:
sequence = []
for i in range (1000,0,-100):
    sequence.append(i)
max_installs_by_mean['SIZE'] = sequence
max_installs_by_mean

,COUNTRY,MEAN_MAX_INSTALLS,SIZE
47,Luxembourg,4.879105e+08,1000
62,Oman,6.050167e+07,900
26,Finland,6.021174e+07,800
81,South Korea,5.321177e+07,700
77,Singapore,5.043485e+07,600
97,Usa,4.871036e+07,500
21,Denmark,4.643685e+07,400
61,Norway,3.377678e+07,300
37,Ireland,2.922904e+07,200
19,Cyprus,2.620449e+07,100


Plotting a map with bubble size according to the sequence of MEAN_MAX_INSTALLS

In [76]:
fig = px.scatter_geo(max_installs_by_mean, locations="COUNTRY", 
                     hover_name="COUNTRY", hover_data=["MEAN_MAX_INSTALLS"], size = 'SIZE',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()

Taking the median of MAX_INSTALLS for each country

In [19]:
max_installs_by_median = pd.DataFrame(edited_df.groupby(["COUNTRY"])["MAX_INSTALLS"].agg("median"))
max_installs_by_median.sort_values("MAX_INSTALLS", ascending = False)
max_installs_by_median = max_installs_by_median.reset_index()
max_installs_by_median.rename(columns={'MAX_INSTALLS': 'MEDIAN_MAX_INSTALLS'}, inplace = True)
max_installs_by_median

,COUNTRY,MEDIAN_MAX_INSTALLS
0,Albania,20544788.0
1,Algeria,162941.0
2,Andorra,10505275.0
3,Argentina,745027.0
4,Armenia,7473048.0
...,...,...
98,Uzbekistan,2080475.0
99,Venezuela,8762.0
100,Vietnam,1813124.0
101,Yemen,10314.0


Taking only the top 10 countries in MEDIAN_MAX_INSTALLS

In [21]:
max_installs_by_median = max_installs_by_median.nlargest(10,['MEDIAN_MAX_INSTALLS'])
max_installs_by_median

,COUNTRY,MEDIAN_MAX_INSTALLS
62,Oman,60585267.0
48,Macedonia,23300899.0
0,Albania,20544788.0
29,Ghana,18902284.0
2,Andorra,10505275.0
54,Montenegro,9165494.5
47,Luxembourg,8944258.0
26,Finland,8359404.5
56,Myanmar,7529986.0
4,Armenia,7473048.0


Plotting a bar plot for MEDIAN_MAX_INSTALLS against COUNTRY

In [23]:
fig = px.bar(max_installs_by_median, x='COUNTRY', y='MEDIAN_MAX_INSTALLS')
fig.show()

It seems that Oman also has a large discrepancy in MEAN_MAX_INSTALLS with the other countries.

Exploring Oman

In [26]:
oman = pd.DataFrame(df[df['COUNTRY'] == 'Oman'])
oman = oman[['APP_NAME','MAX_INSTALLS']]
fig = px.box(oman, x='MAX_INSTALLS')
fig.show()

In [27]:
fig = px.bar(oman, x='APP_NAME', y='MAX_INSTALLS')
fig.show()

The MAX_INSTALLS of all the apps from Oman seems to be more balanced than Luxembourg

Plotting MEDIAN_MAX_INSTALLS in a map with the bubble size in the same ratio as the MEDIAN_MAX_INSTALLS of each country

In [37]:
fig = px.scatter_geo(max_installs_by_median, locations="COUNTRY", 
                     hover_name="COUNTRY", size = 'MEDIAN_MAX_INSTALLS',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()

### Approach 2

Taking the mean of RATING for each country

In [46]:
rating_by_mean = pd.DataFrame(edited_df.groupby(["COUNTRY"])["RATING"].agg("mean"))
rating_by_mean.rename(columns={'RATING': 'MEAN_RATING'}, inplace = True)

Plotting the MEAN_RATING of all countries in a boxplot

In [50]:
fig = px.box(rating_by_mean, x = "MEAN_RATING",points="all")
fig.show()

Plotting the MEAN_RATING of all countries in a histogram

In [51]:
fig = px.histogram(rating_by_mean, x = "MEAN_RATING")
fig.show()

All countries except for one has MEAN_RATINGS larger than 3. 

Checking out the country that has the lowest MEAN_RATING.

In [54]:
lowest_mean_rating = rating_by_mean.nsmallest(1,['MEAN_RATING'])
lowest_mean_rating

,MEAN_RATING
COUNTRY,
Uganda,2.2


Exploring Uganda

In [58]:
uganda = pd.DataFrame(df[df['COUNTRY'] == 'Uganda'])
uganda

,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,5_STAR_RATINGS,...,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
1640,Bin Checker PRO - BIN Card Checker & Verifier,2.2,['Finance'],topselling_paid,10,6,0,2,0,2,...,2300000.0,tricksgum,"Kasubi, Kampala",Uganda,Everyone,False,False,False,456,666


It seems that Uganda only has 1 app and that app has a very low RATING

Taking only the top 10 countries in MEAN_MAX_INSTALLS

In [61]:
rating_by_mean = rating_by_mean.nlargest(10,['MEAN_RATING'])
rating_by_mean = rating_by_mean.reset_index()
rating_by_mean

,index,COUNTRY,MEAN_RATING
0,0,Montenegro,4.800969
1,1,Venezuela,4.670000
2,2,Albania,4.633178
3,3,Costa Rica,4.566433
4,4,Andorra,4.549720
5,5,Nepal,4.540000
6,6,Uruguay,4.539158
7,7,Peru,4.535119
8,8,Tanzania,4.527273
9,9,Ghana,4.511662


In [64]:
sequence = []
for i in range (1000,0,-100):
    sequence.append(i)
rating_by_mean['SIZE'] = sequence
rating_by_mean

,index,COUNTRY,MEAN_RATING,SIZE
0,0,Montenegro,4.800969,1000
1,1,Venezuela,4.670000,900
2,2,Albania,4.633178,800
3,3,Costa Rica,4.566433,700
4,4,Andorra,4.549720,600
5,5,Nepal,4.540000,500
6,6,Uruguay,4.539158,400
7,7,Peru,4.535119,300
8,8,Tanzania,4.527273,200
9,9,Ghana,4.511662,100


Plotting a bar plot for MEAN_MAX_INSTALLS against COUNTRY

In [62]:
fig = px.bar(rating_by_mean, x='COUNTRY', y='MEAN_RATING')
fig.show()

The MEAN_RATING all the countries are quite balanced.

In [74]:
fig = px.scatter_geo(rating_by_mean, locations="COUNTRY", 
                     hover_name="COUNTRY", hover_data=["MEAN_RATING"], size = 'SIZE',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()

### Approach 3

Sorting the dataframe according to COUNTRY first, then MAX_INSTALLS

In [78]:
edited_df = edited_df.sort_values(by=["COUNTRY","MAX_INSTALLS"], ascending = False)
edited_df.reset_index(inplace=True)
edited_df[['COUNTRY','MAX_INSTALLS']]

,COUNTRY,MAX_INSTALLS
0,Zambia,9111
1,Yemen,10314
2,Vietnam,148168255
3,Vietnam,83157347
4,Vietnam,65712634
...,...,...
11577,Argentina,704
11578,Argentina,187
11579,Andorra,10505275
11580,Algeria,162941


Choosing only the highest 10 apps in MAX_INSTALLS for each country and assign it to country_top_10 dataframe

In [80]:
j=0
country_top_10 = pd.DataFrame(edited_df.iloc[:1,])
for i in range (1,edited_df.shape[0]):
    temp = pd.DataFrame(edited_df.iloc[i:i+1,])
    if(j<9):
        country_top_10 = pd.concat([country_top_10,temp])
        j += 1
    if(edited_df.loc[i-1,'COUNTRY']!= edited_df.loc[i,'COUNTRY']):
        j=0

In [81]:
country_top_10.head(30)

,index,APP_NAME,RATING,CATEGORY,COLLECTIONS,RATING_COUNT,1_STAR_RATINGS,2_STAR_RATINGS,3_STAR_RATINGS,4_STAR_RATINGS,...,SIZE,DEVELOPER,DEVELOPER_ADDRESS,COUNTRY,CONTENT_RATING,AD_SUPPORTED,IN_APP_PURCHASES,EDITORS_CHOICE,DAYS_SINCE_UPDATE,DAYS_SINCE_RELEASED
0,1639,Bimostitch Panorama Pro,4.216495,['Photography'],topselling_paid,533,66,5,22,93,...,4000000.0,BCD Vision,"New Avondale, Lusaka, Zambia.",Zambia,Everyone,False,False,False,20,1328
1,13201,SmartDaddy: Parental Control Child Protection,3.900000,['Parenting'],topgrossing,42,8,4,0,0,...,23000000.0,SmartDaddy,SmartDaddy Parental Control,Yemen,Everyone,False,True,False,168,1698
2,3498,DEAD TARGET: Zombie Games 3D,4.551771,['Action'],topselling_free,2195758,103200,47680,96892,234540,...,172000000.0,VNG GAME STUDIOS,"182 Le Dai Hanh, Ward 15, District 11, HCMC",Vietnam,Mature 17+,True,True,False,11,2822
3,343,"AZ Screen Recorder - Video Recorder, Livestream",4.448757,['Video Players & Editors'],topgrossing,1578130,123923,39222,62246,132029,...,13000000.0,AZ Screen Recorder,Vietnam,Vietnam,Everyone,True,True,False,128,2674
4,11010,Piano +,4.334832,['Music & Audio'],topselling_free,770443,67123,21176,40587,99279,...,22000000.0,FunAIs JSC,"5th Floor, 62 Tran Quang Khai, Phuong Tan Dinh...",Vietnam,Everyone,True,True,False,41,3636
5,10903,"Phone 13 Launcher, OS 15",4.705272,['Tools'],topselling_free,701683,33147,5848,10907,34766,...,43000000.0,SaS Developer,"Thai Ha, Dong Da, Ha Noi",Vietnam,Everyone,True,True,False,50,1586
6,10096,Ninja warrior: legend of adventure games,4.437097,['Adventure'],topselling_free,372706,30642,13184,11378,24622,...,44000000.0,TOH Games,"HN, VN",Vietnam,Teen,True,True,False,24,873
7,14703,Thief Puzzle - Can you steal it ?,4.105797,['Puzzle'],topselling_free,122891,17800,4625,6044,12641,...,84000000.0,WEEGOON,"143 NGUYEN TUAN ROAD, THANH XUAN, HA NOI",Vietnam,Everyone,True,True,False,3,408
8,16170,Weather forecast,4.722475,['Weather'],topselling_free,897152,24600,8278,22965,79767,...,11000000.0,smart-pro android apps,"Ha Noi, Viet Nam",Vietnam,Everyone,True,True,False,490,1587
9,7915,Launcher iOS 15,4.781462,['Personalization'],topselling_free,2101684,52937,15357,40309,120709,...,45000000.0,LuuTinh Developer,"Hanoi, Vietnam",Vietnam,Everyone,True,False,False,209,1277


Taking the mean of RATING for each country

In [82]:
rating_by_mean = pd.DataFrame(country_top_10.groupby(["COUNTRY"])["RATING"].agg("mean"))
rating_by_mean = rating_by_mean.reset_index()
rating_by_mean

,COUNTRY,RATING
0,Albania,4.633178
1,Algeria,3.555556
2,Argentina,4.409667
3,Armenia,4.370000
4,Australia,4.312364
...,...,...
85,Usa,4.118186
86,Uzbekistan,4.051272
87,Vietnam,4.490154
88,Yemen,3.900000


Taking only the top 10 countries in RATING

In [84]:
rating_by_mean = rating_by_mean.nlargest(10,['RATING'])
sequence = []
for i in range (1000,0,-100):
    sequence.append(i)
rating_by_mean['SIZE'] = sequence
rating_by_mean


,COUNTRY,RATING,SIZE
46,Montenegro,4.800969,1000
0,Albania,4.633178,900
55,Peru,4.619394,800
84,Uruguay,4.569216,700
58,Portugal,4.525657,600
20,Dominican Republic,4.511111,500
7,Belarus,4.503265,400
53,Oman,4.500407,300
72,Spain,4.492090,200
61,Romania,4.491281,100


Plotting a bar plot for mean RATING against COUNTRY

In [90]:
fig = px.bar(rating_by_mean, x='COUNTRY', y='RATING')
fig.show()

The mean rating is quite balance across the top 10 countries

Plotting the mean RATING of the 10 countries in a map

In [89]:
fig = px.scatter_geo(rating_by_mean, locations="COUNTRY", 
                     hover_name="COUNTRY", hover_data = ["RATING"], size = 'SIZE',locationmode = 'country names')
fig.show()

### Approach 4

Taking the median of MAX_INSTALLS of each country and choosing only the highest 10 apps

In [95]:
max_installs_by_median = pd.DataFrame(country_top_10.groupby(["COUNTRY"])["MAX_INSTALLS"].agg("median"))
max_installs_by_median.rename(columns={'MAX_INSTALLS': 'MEDIAN_MAX_INSTALLS'}, inplace = True)
max_installs_by_median = max_installs_by_median.nlargest(10,['MEDIAN_MAX_INSTALLS'])
max_installs_by_median = max_installs_by_median.reset_index()

In [96]:
sequence = []
for i in range (1000,0,-100):
    sequence.append(i)
max_installs_by_median['SIZE'] = sequence
max_installs_by_median

,COUNTRY,MEDIAN_MAX_INSTALLS,SIZE
0,Usa,9.142739e+09,1000
1,South Korea,6.545143e+08,900
2,Singapore,4.846291e+08,800
3,France,3.160152e+08,700
4,Finland,2.534177e+08,600
5,United Kingdom,2.230780e+08,500
6,Hong Kong,2.198699e+08,400
7,China,1.912359e+08,300
8,Germany,1.873334e+08,200
9,Turkey,1.829784e+08,100


Plotting a bar plot for MEDIAN_MAX_INSTALLS against COUNTRY

In [97]:
fig = px.bar(max_installs_by_median, x='COUNTRY', y='MEDIAN_MAX_INSTALLS')
fig.show()

    The MEDIAN_MAX_INSTALLS of Usa has an extremely large discrepancy with all the other countries.
    Also, we can see that Singapore is in the third place.

Plotting the MEDIAN_MAX_INSTALLS of the 10 countries in a map

In [99]:
fig = px.scatter_geo(max_installs_by_median, locations="COUNTRY", 
                     hover_name="COUNTRY", hover_data=["MEDIAN_MAX_INSTALLS"], size = 'SIZE',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()

Taking the mean of MAX_INSTALLS of each country and choosing only the highest 10 apps

In [104]:
max_installs_by_mean = pd.DataFrame(country_top_10.groupby(["COUNTRY"])["MAX_INSTALLS"].agg("mean"))
max_installs_by_mean.rename(columns={'MAX_INSTALLS': 'MEAN_MAX_INSTALLS'}, inplace = True)
max_installs_by_mean = max_installs_by_mean.nlargest(10,['MEAN_MAX_INSTALLS'])
max_installs_by_mean = max_installs_by_mean.reset_index()

In [105]:
sequence = []
for i in range (1000,0,-100):
    sequence.append(i)
max_installs_by_mean['SIZE'] = sequence
max_installs_by_mean
max_installs_by_mean

,COUNTRY,MEAN_MAX_INSTALLS,SIZE
0,Usa,8.837927e+09,1000
1,South Korea,8.645879e+08,900
2,Singapore,7.578918e+08,800
3,Luxembourg,4.879105e+08,700
4,France,3.091444e+08,600
5,Finland,2.760915e+08,500
6,India,2.496827e+08,400
7,China,2.415462e+08,300
8,Turkey,2.173386e+08,200
9,Hong Kong,2.158338e+08,100


Plotting a bar plot for MEAN_MAX_INSTALLS against COUNTRY

In [106]:
fig = px.bar(max_installs_by_mean, x='COUNTRY', y='MEAN_MAX_INSTALLS')
fig.show()

Usa, South Korea, Singapore, France, Finland, China, Turkey and Hong Kong all remains on the list.
Luxembourg is now in the top 10 list, which is expected since both Viber and Skype has a very high MAX_INSTALLS value.

Plotting the MEAN_MAX_INSTALLS of the 10 countries in a map

In [109]:
fig = px.scatter_geo(max_installs_by_mean, locations="COUNTRY", 
                     hover_name="COUNTRY", hover_data=["MEAN_MAX_INSTALLS"], size = 'SIZE',locationmode = 'country names')
fig.update_layout({
    'geo': {
        'resolution': 50
    }
})
fig.show()